# Event Centering 2021-04-14

Goal of this notebook is to test out functions for event centering on 1 day, 1 neuron. And then, 1 day, all neurons. And then all days all neurons. Plan is to save out as dicts (nested) to used for later analysis specific to delay period analysis.

### Libs & Utils

In [2]:
# libraries
import sys; sys.path.insert(0, '..') # if you don't find it here, look one above
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.io as spio
import pickle
from tqdm import tqdm
from scipy import stats
# stored one repo up in my fork of Spykes
# from spykes.spykes.plot.neurovis import NeuroVis
from io_utils import *
from plotting_utils import * 


# settings
%load_ext autoreload
%autoreload 2
sns.set_context("talk")

### Load session

Here, I'm loading in one session (08-06-2019) that has 4 cells 

In [3]:
sess_name = 'W122_08_06_2019_1_fromSD'

base_path  = 'Y:\jbreda\ephys\post_sort_analysis\sorted_pre_bdata' 
beh_mat   = 'protocol_info.mat'
spks_mat  = 'ksphy_clusters_foranalysis.mat'

# create paths
sess_path = os.path.join(base_path, sess_name)
beh_path  = os.path.join(sess_path, beh_mat)
spks_path = os.path.join(sess_path, spks_mat)
# fig_save_path = os.path.join(os.getcwd(), 'figures', 'neurovis', sess_name)

# load & wrangle
beh_df, spks_dict = load_and_wrangle(beh_path, spks_path, overwrite=False)
neuron1 = spks_dict['spk_times'][0]

In [7]:
neuron1_spks = split_spks_by_trial(neuron1, beh_df)

In [8]:
neuron1_spks.keys()

dict_keys(['nums', 'times', 'spks'])

Stuck here because I'm not sure how to append to list. Might need to append to initialzied dic instead?

In [32]:
test = align_neuron_to_events(beh_df, neuron1_spks)

I want to give a function the spks_dict and beh_df for a section. I then want it to create a dictonary where the first level of keys is a neuron. and then the second level of keys is the event aligned information for that neuron. This dict will be saved out and loaded in so raw spk times are no longer needed.

In [11]:
len(spks_dict['spk_times'])

4

In [27]:
def event_align_session(spks_dict, beh_df, sess_path, overwrite=False, delay_mode=True):
    """ Function for alining spike times from each neuron in a session to relevant events
    and saving out as a dictionary
    
    Params:
    ------
    
    Returns:
    --------
    """
    
    
    # check to see if already there, load in if so
    file_name = 'event_aligned_spks.pkl'
    if os.path.exists(os.path.join(sess_path, file_name)) and overwrite==False:
        
        print('loading from file...')
        
        with open(os.path.join(sess_path, file_name), 'rb') as fh:
            session_aligned = pickle.load(fh)
    else:
        
        print('no file found, running alignment for session')
        
        # initialize information
        session_spks = spks_dict['spk_times']
        n_neurons = len(session_spks)
        session_aligned = {}

        for ineuron in range(n_neurons):

            # TODO saving out additional info from this might be nice at some point
            trial_spks = split_spks_by_trial(session_spks[ineuron], beh_df)
            neuron_aligned = align_neuron_to_events(beh_df, trial_spks, delay_mode=delay_mode)
            session_aligned[ineuron] = neuron_aligned
            
        # save out
        output = open(os.path.join(sess_path, file_name), 'wb')
        pickle.dump(session_aligned, output)
        output.close()

        
    return session_aligned
        
        
        
        
    

In [30]:
k = event_align_session(spks_dict, beh_df, sess_path)

no file found, running alignment for session


In [34]:
with open(os.path.join(sess_path,'event_aligned_spks.pkl'), 'rb') as fh:
    j = pickle.load(fh)

In [45]:
j[0]['aud1on'] == k[0]['aud1on']

[array([-0.19569808, -0.14606717, -0.13356778, -0.09890281, -0.06120466,
        -0.04717201, -0.04490545, -0.00807392,  0.02745767,  0.0323241 ,
         0.05052321,  0.08308829,  0.08682144,  0.0882547 ,  0.10332063,
         0.13791894,  0.15471811,  0.19124966,  0.21864832,  0.25477989,
         0.26364612,  0.27004581,  0.28667833,  0.327343  ,  0.35614159,
         0.35887479,  0.36097469,  0.37624061,  0.37714057,  0.41983848,
         0.45900323,  0.47343586,  0.52266678,  0.53739939,  0.55356527,
         0.57989732,  0.58733029]),
 array([-0.37207007, -0.34703796, -0.23321019,  0.15957059,  0.20403509,
         0.2581991 ,  0.28673104,  0.31019656,  0.31226312,  0.31332974]),
 array([-0.39395074, -0.37345174, -0.35548595, -0.34615308,  0.41004327,
         0.42957564,  0.44250834,  0.44717478,  0.48320635,  0.48897274,
         0.50620523,  0.53883697,  0.56040258,  0.56420239,  0.56990211,
         0.58070158,  0.5810349 ]),
 array([-0.36584151, -0.36140839, -0.33827619, -0.

Stopped here--- TODO:



* expand code written above to multiple neurons
* saving out a sessions aligned spk dict (nested for each neuron) will be good. Can just load that in for analyses.

## non function code

In [59]:
# these are names in the beh_df
df_event = ['aud1_on', 'aud1_off', 'aud2_on', 'aud2_off', 'aud1_on']
names = ['aud1on', 'aud1off', 'aud2on', 'aud2off', 'trial_all']
windows = [[-400, 600], [-500, 500], [-400, 600], [-500, 500], [-100, 5000]]


aligned_spk_times = {}

for event,name,window in zip(df_event,names,windows):
    rowsL = []
    aligned_spk_times[name] = []
    
    for itrial, row in beh_df.iterrows():
        
        trial_spks = neuron1_spks['trial_spks'][itrial]

        # grab alignment time
        align_time = row[event]
        
        # create windows
        start = align_time + (window[0] * 0.001)
        stop  = align_time + (window[1] * 0.001)  
        
         # grab spks in the window
        spks_in_window = trial_spks[np.logical_and(trial_spks > start, trial_spks < stop)]
        event_aligned = spks_in_window - align_time
        
        # append
        aligned_spk_times[name].append(event_aligned)
        

In [102]:
# delay

def delay_align(beh_df, trial_spks):
    delay_windows = [[0, 2000],[-200, 2400], [0, 4000], [-200, 4400]]
    delay_names = ['delay2s', 'trial2s', 'delay4s', 'trial4s']


L =[[],[],[],[]]


    for itrial, row in beh_df.iterrows():

        trial_spks = neuron1_spks['trial_spks'][itrial]
        d_align_time = row['aud1_off']
        t_align_time = row['aud1_on']

        if row['delay'] == 2:

            d = 0
            d_start  = d_align_time + (delay_windows[d][0] * 0.001)
            d_stop   = d_align_time + (delay_windows[d][1] * 0.001)

            spks_in_delay = trial_spks[np.logical_and(trial_spks > d_start, trial_spks < d_stop)]
            delay_aligned = spks_in_delay - d_align_time
            L[d].append(delay_aligned)

            t = 1
            t_start  = t_align_time + (delay_windows[t][0] * 0.001)
            t_stop   = t_align_time + (delay_windows[t][1] * 0.001)

            spks_in_trial = trial_spks[np.logical_and(trial_spks > t_start, trial_spks < t_stop)]
            trial_aligned = spks_in_trial - t_align_time
            L[t].append(trial_aligned)

        elif row['delay'] == 4:

            d = 2 
            d_start  = d_align_time + (delay_windows[d][0] * 0.001)
            d_stop   = d_align_time + (delay_windows[d][1] * 0.001)

            spks_in_delay = trial_spks[np.logical_and(trial_spks > d_start, trial_spks < d_stop)]
            delay_aligned = spks_in_delay - d_align_time
            L[d].append(delay_aligned)

            t = 3
            t_start  = t_align_time + (delay_windows[t][0] * 0.001)
            t_stop   = t_align_time + (delay_windows[t][1] * 0.001)

            spks_in_trial = trial_spks[np.logical_and(trial_spks > t_start, trial_spks < t_stop)]
            trial_aligned = spks_in_trial - t_align_time
            L[t].append(trial_aligned)


    delay_info = dict(zip(delay_names, L))